# Retrival

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
caminhos = [
    "samples/LangChain.pdf",
    "samples/LangChain.pdf",
    ]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

### Modificando Metadata

In [ ]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

In [ ]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

## Similarity Search

In [ ]:
pergunta = 'O que é a HuggingFace?'

docs = vectordb.similarity_search(pergunta, k=3)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

## Max Marginal Relevance Search

In [ ]:
docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

## Filtragem

In [ ]:
docs = vectordb.similarity_search(
    pergunta, 
    k=3,
    filter={'source': 'LangChain.pdf'})
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

## LMM Aided Retrival

In [3]:
from langchain_ollama.llms import OllamaLLM
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [4]:
metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Deve ter o valor de: \
            LangChain.pdf',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto se origina',
        type='integer'
    ),

]

In [ ]:
document_description = 'LangChain'
llm = OllamaLLM(model="llama2")
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = retriever.invoke(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')